## 1.You can only use a sample to do your experiment. That's why we need to understand the difference between population and sample, and then use confidence interval to test our hypothesis.

## 2.Use confidence interval to attempt to determine how accurate our sample mean estimation

In [2]:
import numpy as np 
import pandas as pd

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 30 19:41:59 2023

@author: jrjol
"""
from polygon import RESTClient

client = RESTClient(api_key="ocunxnOqC0pnltRqT3VkOiKeCmPE49L7")

import requests

url = "https://api.polygon.io/v2/aggs/ticker/SPY/range/1/day/2015-10-22/2024-01-07?adjusted=true&sort=asc&limit=5000&apiKey=ocunxnOqC0pnltRqT3VkOiKeCmPE49L7"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)
    
import pandas as pd

results = data["results"]
df = pd.DataFrame(results)

# convert the time from mili-second to datetime 
df['t'] = pd.to_datetime(df['t'], unit='ms')

df.index = df['t']
spy_table = df

spy_total = spy_table[['o','c']]
#calculate log returns
spy_log_return = np.log(spy_total.c).diff().dropna()
print('Population mean:', np.mean(spy_log_return))

print('Population standard deviation:',np.std(spy_log_return))

{'ticker': 'SPY', 'queryCount': 2065, 'resultsCount': 2065, 'adjusted': True, 'results': [{'v': 175150531.0, 'vw': 204.1272, 'o': 202.98, 'c': 205.26, 'h': 205.51, 'l': 202.81, 't': 1445486400000, 'n': 472770}, {'v': 144798285.0, 'vw': 206.9993, 'o': 207.25, 'c': 207.51, 'h': 207.95, 'l': 206.3, 't': 1445572800000, 'n': 415961}, {'v': 69222240.0, 'vw': 207.0612, 'o': 207.3, 'c': 207, 'h': 207.37, 'l': 206.56, 't': 1445832000000, 'n': 228237}, {'v': 78111647.0, 'vw': 206.3805, 'o': 206.2, 'c': 206.6, 'h': 207, 'l': 205.79, 't': 1445918400000, 'n': 273129}, {'v': 136067349.0, 'vw': 207.6229, 'o': 207, 'c': 208.95, 'h': 208.98, 'l': 206.21, 't': 1446004800000, 'n': 433475}, {'v': 90726308.0, 'vw': 208.7627, 'o': 208.35, 'c': 208.83, 'h': 209.27, 'l': 208.21, 't': 1446091200000, 'n': 276181}, {'v': 131298016.0, 'vw': 208.6129, 'o': 209.06, 'c': 207.93, 'h': 209.44, 'l': 207.74, 't': 1446177600000, 'n': 353836}, {'v': 86644803.0, 'vw': 209.6025, 'o': 208.32, 'c': 210.39, 'h': 210.62, 'l': 2

In [5]:
print('10 days sample returns:', np.mean(spy_log_return.tail(10)))

print('10 days sample standard deviation:', np.std(spy_log_return.tail(10)))

print('1000 days sample returns:', np.mean(spy_log_return.tail(1000)))

print('1000 days sample standard deviation:', np.std(spy_log_return.tail(1000)))


10 days sample returns: -7.263560745762732e-05
10 days sample standard deviation: 0.004833495321395339
1000 days sample returns: 0.0003547046321690583
1000 days sample standard deviation: 0.01439422066108379


In [6]:
#apply the formula above to calculate confidence interval(1.96 means 95%)
bottom_1 = np.mean(spy_log_return.tail(10))-1.96*np.std(spy_log_return.tail(10))/(np.sqrt(len((spy_log_return.tail(10)))))
upper_1 = np.mean(spy_log_return.tail(10))+1.96*np.std(spy_log_return.tail(10))/(np.sqrt(len((spy_log_return.tail(10)))))
bottom_2 = np.mean(spy_log_return.tail(1000))-1.96*np.std(spy_log_return.tail(1000))/(np.sqrt(len((spy_log_return.tail(1000)))))
upper_2 = np.mean(spy_log_return.tail(1000))+1.96*np.std(spy_log_return.tail(1000))/(np.sqrt(len((spy_log_return.tail(1000)))))
#print the outcomes
print('10 days 95% confidence inverval:', (bottom_1,upper_1))

print('1000 days 95% confidence inverval:', (bottom_2,upper_2))


10 days 95% confidence inverval: (-0.0030684670454314924, 0.0029231958305162378)
1000 days 95% confidence inverval: (-0.0005374584074996976, 0.0012468676718378144)


### we usually use 90%, 95% and 99% as the confidence level of a confidence interval. The critical values for these three confidence levels are 1.64, 1.96, and 2.32 respectively.

### 'Three sigma rule' or '68-95-99.7' rule associated with normal distribution. 

### Does this means we assume the mean of sample follows a normal distribution? The answer is yes. This assumption is supported by central limit theorem.

# Hypothesis Testing

In [7]:
mean_1000 = np.mean(spy_log_return.tail(1000))
std_1000 = np.std(spy_log_return.tail(1000))
mean_10 = np.mean(spy_log_return.tail(10))
std_10 = np.std(spy_log_return.tail(10))
s = pd.Series([mean_10,std_10,mean_1000,std_1000],index = ['mean_10', 'std_10','mean_1000','std_1000'])
print(s)

mean_10     -0.000073
std_10       0.004833
mean_1000    0.000355
std_1000     0.014394
dtype: float64


In [10]:
bottom = 0 - 1.64*std_1000/np.sqrt(1000)
upper = 0 + 1.64*std_1000/np.sqrt(1000)
print((bottom, upper))

(-0.000746503767886102, 0.000746503767886102)


In [9]:
bottom = 0 - 1.96*std_1000/np.sqrt(1000)
upper = 0 + 1.96*std_1000/np.sqrt(1000)
print((bottom, upper))

(-0.000892163039668756, 0.000892163039668756)


# Z score
## The higher the Z score is, the further the tested value is from the hypothesized value (which is 0 in this example). 

In [11]:
print(np.sqrt(1000)*(mean_1000 - 0)/std_1000)

0.7792533966767747


### If we use 95% confidence interval, we need a p-value less than 0.025 to reject the null hypothesis. However, now our p-value is 0.032243, which is greater than 0.025

In [12]:
import scipy.stats as st
print((1 - st.norm.cdf(1.8488)))

0.03224334868551282


In [14]:
mean_1200 = np.mean(spy_log_return.tail(1200))
std_1200 = np.std(spy_log_return.tail(1200))
z_score = np.sqrt(1200)*(mean_1200 - 0)/std_1200
print('z-score = ',z_score)

p_value = (1 - st.norm.cdf(z_score))
print('p_value = ',p_value)


z-score =  1.0538672794081863
p_value =  0.14597184266899266


### our p-value is 0.014, and it's a two-tail test, so our confidence level of the interval is 1-(0.014 * 2) = 0.972. We can say at most with 97.2% confidence interval, we can claim that the population mean is not zero. 